In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/processed/0.1-initial-exploration.csv',index_col=False)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.loc[:,'positionOrder'] = pd.to_numeric(df.loc[:, 'positionOrder'], errors='coerce')
df.loc[:,'ageAtRace'] = pd.to_numeric(df.loc[:, 'ageAtRace'], errors='coerce')
df.loc[:,'ageAtDebut'] = pd.to_numeric(df.loc[:, 'ageAtDebut'], errors='coerce')
df.loc[:,'yearsExperience'] = pd.to_numeric(df.loc[:, 'yearsExperience'], errors='coerce')

**Sort by year and round**

In [ ]:
df = df.sort_values(by=['year','round'])

In [ ]:
df['Finished'] = df['position'] != '\\N'

**Carry forward last Driver Standings statistics**

In [ ]:
df['driverStandingsPoints'] = df.groupby(['driverId'])['driverStandingsPoints'].transform(lambda x: x.ffill())
df['driverStandingsPosition'] = df.groupby(['driverId'])['driverStandingsPosition'].transform(lambda x: x.ffill())
df['driverStandingsWins'] = df.groupby(['driverId'])['driverStandingsWins'].transform(lambda x: x.ffill())

**Carry forward last Constructor Standings statistics**

In [ ]:
df['constructorStandingsPoints'] = df.groupby(['constructorId'])['constructorStandingsPoints'].transform(lambda x: x.ffill())
df['constructorStandingsPosition'] = df.groupby(['constructorId'])['constructorStandingsPosition'].transform(lambda x: x.ffill())
df['constructorStandingsWins'] = df.groupby(['constructorId'])['constructorStandingsWins'].transform(lambda x: x.ffill())

**Drop unwanted columns**

In [ ]:
df = df.drop(['resultId','raceId','rank','driverStandingsId','constructorStandingsId','position','positionText','pointsGained',
'laps','nationality','time_x','milliseconds','fastestLapSpeed','fastestLapTime','fastestLap','statusId'
,'circuitName','dateOfDebut','status','driverStandingsId'], 1)

**Columns with null values**

In [ ]:
print('Columns with null values:\n', df.columns[df.isna().any()].tolist())

**Replace Nans with 0 or -1 depending on the column**

In [ ]:
df[['racesWon','racesRetired','racesFinished','polePositions','racesWonByConstructor','racesRetiredByConstructor']] = df[['racesWon','racesRetired','racesFinished','polePositions','racesWonByConstructor','racesRetiredByConstructor']].fillna(value=0)
df[['lastRaceRank','previousRaceGridStart','previousRacePosition']] = df[['lastRaceRank','previousRaceGridStart','previousRacePosition']].fillna(value=-1)

**Replace missing percentage of best qualifying time**

In [ ]:
#df['percentageOfBestQuali'] = df.groupby(['driverId','year'])['percentageOfBestQuali'].transform(lambda x: x.fillna(x.mean()))
df['percentageOfBestQuali'] = df.groupby(['driverId','year'])['percentageOfBestQuali'].transform(lambda x: x.ffill())
df['percentageOfBestQuali'] = df.groupby(['year','round'])['percentageOfBestQuali'].transform(lambda x: x.fillna(x.mean()))
df['percentageOfBestQuali'] = df.groupby(['year','round'])['percentageOfBestQuali'].transform(lambda x: x.fillna(100))

**Chech which columns still have missing values**

In [ ]:
print('Columns with null values:\n', df.columns[df.isna().any()].tolist())
df[df.isnull().any(axis=1)].head(20)

Only 6 rows have missing constructor standings information now, all of which are from 1956 or before.

**Fill missing rank (ordering of drivers by previous race fastest lap during race) values with 0**

In [ ]:
df['lastRaceRank'] = df['lastRaceRank'].replace({'\\N': 0})
df['lastRaceRank'] = df['lastRaceRank'].astype(str).astype(int)

**Replace grid starting position 0 (pit lane start) with the last starting position of that race +1**

In [ ]:
df['gridStart'] = df.groupby(['year','round'])['gridStart'].transform(lambda x: x.replace(to_replace = 0, value = x.max()+1))

In [ ]:
df['racingAtHome'] = df["racingAtHome"].astype(int)

In [ ]:
df.info()

**Add artificial entry for last race (round 22) of 2021 for Mazepin. He couldnt race because of covid at the time**

In [ ]:
new_row = {'driverId':853, 'constructorId':210, 'gridStart':20, 'positionOrder':20, 'driverRef': 'mazepin',
 'year':2021, 'round':22, 'circuitId':77, 'date':'2021-12-12', 'ageAtRace':22, 'ageAtDebut':22,
  'yearsExperience':0, 'racingAtHome':0, 'driverStandingsPoints':0, 'driverStandingsPosition':19,
  'driverStandingsWins':0, 'lastRaceRank':19, 'constructorStandingsPoints':0, 'constructorStandingsPosition':10,
  'constructorStandingsWins':0, 'previousRaceGridStart':19, 'previousRacePosition': 18, 'racesWon':0,
  'racesRetired':4, 'racesFinished': 2, 'polePositions': 0, 'racesWonByConstructor': 0, 'racesRetiredByConstructor': 46,
  'percentageOfBestQuali': 115}
#append row to the dataframe
df = df.append(new_row, ignore_index=True)

In [ ]:
df.reset_index(drop=True)
df.to_csv('../data/processed/0.3-feature-processing.csv')

In [ ]:
df.tail()